<span style="font-family: Arial,Helvetica,sans-serif; font-size: medium">
<div align="center"><span style="font-size: x-large; color:#0000FF"><b>Métodos Numéricos II</b></span></div>
<div align="center"><span style="font-size: large; color:#0000FF"><b>Segundo de Grado en Matemáticas - Curso 2021/22</b></span></div>
<div align="center"><span style="font-size: medium; color:#0000FF"><b>Facultad de Ciencias de la Universidad de Málaga</b></span></div>
<div align="center"><span style="font-size: small; color:#0000FF"><b>Dpto. de Análisis Matemático, Estadística e Investigación Operativa, y Matemática Aplicada</b></span></div>
<div align="center"><span style="font-size: small; color:#0000FF"><b>Profs. María López Fernández y Francisco José Palma Molina (Área Conocimiento de Matemática Aplicada)</b></span></div>
<div align="center"><span style="font-size: medium; color:#FF0000"><b>Práctica número 7</b></span></div></span>

In [1]:
from algoritmos import *

El objetivo de esta práctica es desarrollar funciones <span style="font-family: Courier,monospace">Python</span> para resolver sistemas lineales mediante los <b>métodos iterativos clásicos de Jacobi, de Gauss-Seidel y de relajación</b>.

Recordamos que, en general, los métodos iterativos para resolver un sistema lineal compatible y determinado de $n$ ecuaciones con $n$ incógnitas $A\,X=B$ (donde los datos del problema son $A\in\mathcal{M}_n(\mathbb{K})$ inversible y $B\in\mathbb{K}^n$, y la incógnita es $X\in\mathbb{K}^n$), se basan en construir una matriz $C\in\mathcal{M}_n(\mathbb{K})$ y un vector $V\in\mathbb{K}^n$ tales que
$$
A\,X = B \quad \Leftrightarrow \quad X = C\,X +V\,.
$$
<br>
A partir de esta formulación equivalente del problema, se construye una sucesión $\{X_k\}_{k\in\mathbb{N}}$ de aproximaciones de la solución de la forma siguiente:

$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ X_{k+1} = C\,X_k + V\,, \quad k=0,1,2,\ldots. \end{array} \right.
$$

<br>
Es claro que esta construcción de la sucesión permite asegurar que en caso de convergencia lo hará hacia la única solución del problema dado, y sabemos que esa convergencia se produce si y solo si $\varrho(C)<1$.
<br>

Para la construcción de la matriz $C$ y del vector $V$, se parte de una descomposición de la matriz $A=M-N$, donde $M,N\in\mathcal{M}_n(\mathbb{K})$, con $M$ inversible: se escribe entonces

$$
A\,X = B \quad \Leftrightarrow \quad X = M^{-1}\,N\,X + M^{-1}\,B\,,
$$

con lo que $C=M^{-1}\,N$ y $V=M^{-1}\,B$.

<br>
Los métodos iterativos clásicos utilizan la descomposición de la matriz $A$ en la forma

$$
A = D - E - F\,,
$$

donde las matrices $D,E,F\in\mathcal{M}_n(\mathbb{K})$ son, respectivamente, diagonal, estrictamente triangular inferior y estrictamente triangular superior. Hacemos siempre la hipótesis que los elementos diagonales de $A$ son no nulos, lo que asegura que la matriz $D$ siempre es inversible.


El <b>método iterativo de Jacobi</b> se basa en la elección $M=D$ y $N=E+F$, con lo que se tiene que $C=D^{-1}\,(E+F)$ (esta matriz se suele notar mediante $J$) y $V=D^{-1}\,B$. La sucesión generada $\{X_k\}_{k\in\mathbb{N}}$ viene dada por
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ X_{k+1} = D^{-1}\,(E+F)\,X_k + D^{-1}\,B\,, \quad k=0,1,2,\ldots, \end{array} \right.
$$
o equivalentemente
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ D\,X_{k+1} = B + (E+F)\,X_k\,, \quad k=0,1,2,\ldots. \end{array} \right.
$$
<br>

Si ponemos $X_k=(x_i^{(k)})_{k=1}^n$, entonces
$$
x_i^{(k+1)} = \frac{1}{a_{i,i}}\,\left( b_i - \sum_{j=1\,,\,j\ne i}^n a_{i,j}\,x_j^{(k)} \right)\,.
$$

<span style="color:#FF0000"><b>Ejercicio 1.</b></span> Elaborar un programa de nombre <span style="font-family: Courier,monospace">jacobi()</span> que implemente el algoritmo del <b>método iterativo de Jacobi</b>.

<span style="color:#FF0000"><b>Observación.</b></span> En todos los programas que siguen, los mismos llevan como parámetros de entrada la matriz $A$, el segundo miembro $B$, el vector $X_0$ con el que iniciar las iteraciones, el número máximo de iteraciones a realizar y la tolerancia del test de parada (que es la norma infinito de la diferencia entre dos iteraciones sucesivas); en el caso del método de relajación, también hay que dar el parámetro de relajación $\omega$.

In [ ]:
def jacobi(A, B, XOLD, itermax, tol):
    m, n = shape(A)
    p, q = shape(B)
    r, s = shape(XOLD)
    if m != n or n != p or q != 1 or n != r or s != 1 or min(abs(diag(A))) < 1e-200:
        return False, 'ERROR jacobi: no se resuelve el sistema.'
    k = 0
    error = 1.
    while k < itermax and error >= tol:
        k = k+1
        XNEW = array(B)
        for i in range(n):
            if i != 0:
                XNEW[i, 0] -= A[i, :i]@XOLD[:i, 0]
            if i != n-1:
                XNEW[i, 0] -= A[i, i+1:]@XOLD[i+1:, 0]
            XNEW[i, 0] = XNEW[i, 0]/A[i, i]
        error = norma_vec(XNEW - XOLD, inf)
        XOLD = array(XNEW)
    print('Iteración: k = ', k)
    print('Error absoluto: error = ', error)
    if k == itermax and error >= tol:
        return False, 'ERROR jacobi: no se alcanza convergencia.'
    else:
        print('Convergencia numérica alcanzada: jacobi.')
        return True, XNEW

<span style="color:#FF0000"><b>Ejercicio 2.</b></span> Resolver mediante el método iterativo de Jacobi un sistema lineal $A\,X=B$, cuya matriz de coeficientes $A$ (del tamaño que se quiera) es tridiagonal, con elementos diagonales iguales a 2 y elementos sub-diagonales y supra-diagonales iguales a $-1$ (esta matriz es definida positiva); sabemos que hay convergencia en este caso. Tomamos como segundo miembro $B$ un vector cuyas componentes son las sumas de las respectivas filas de la matriz $A$, lo que nos asegura que la solución exacta del sistema $X$ es el vector con todas las componentes iguales a 1. Tomamos como vector inicial $X_0$ el vector nulo, establecemos un número máximo de iteraciones de 1000 y un valor para la constante de tolerancia de $10^{-5}$.

In [2]:
# Ejemplos de método de Jacobi
print('Ejemplo de Jacobi.')
n = 5
A = 2*eye(n) - eye(n, k=-1) - eye(n, k=1)
print('Matriz: A = ', A)
B = reshape(sum(A, axis=1), (n,1))
print('Segundo miembro: B = ', B)
X_0 = zeros((n, 1))
print('Vector inicial: X_0 = ', X_0)
exito, X = jacobi(A, B, X_0, 1000, 1e-5)
if exito:
    print('Solución aproximada: X_k = ', X)
    print('Comprobación: ||B-A X||_2 =',norm(B-A@X, 2))
else:
    print(X)

Ejemplo de Jacobi.
Matriz: A =  [[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
Segundo miembro: B =  [[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
Vector inicial: X_0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Iteración: k =  74
Error absoluto: error =  7.945943831355606e-06
Convergencia numérica alcanzada: jacobi.
Solución aproximada: X_k =  [[0.99998411]
 [0.99997616]
 [0.99996822]
 [0.99997616]
 [0.99998411]]
Comprobación: ||B-A X||_2 = 1.946350791163682e-05


El <b>método iterativo de Gauss-Seidel</b> se basa en la elección $M=D-E$ y $N=F$, con lo que se tiene que $C=(D-E)^{-1}\,F$ (esta matriz se suele notar mediante $\mathcal{L}_1$) y $V=(D-E)^{-1}\,B$. La sucesión generada $\{X_k\}_{k\in\mathbb{N}}$ viene dada por
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ X_{k+1} = (D-E)^{-1}\,F\,X_k + (D-E)^{-1}\,B\,, k=0,1,2,\ldots, \end{array} \right.
$$
o equivalentemente
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ D\,X_{k+1} = B + E\,X_{k+1} + F\,X_k\,, \quad k=0,1,2,\ldots. \end{array} \right.
$$
<br>

Si ponemos $X_k=(x_i^{(k)})_{k=1}^n$, entonces
$$
x_i^{(k+1)} = \frac{1}{a_{i,i}}\,\left( b_i - \sum_{j=1}^{i-1} a_{i,j}\,x_j^{(k+1)} - \sum_{j=i+1}^n a_{i,j}\,x_j^{(k)} \right)\,.
$$

<span style="color:#FF0000"><b>Ejercicio 3.</b></span> Elaborar un programa de nombre <span style="font-family: Courier,monospace">gauss_seidel()</span> que implemente el algoritmo del <b>método iterativo de Gauss-Seidel</b>.

In [ ]:
# Esta en algortimos.py

<span style="color:#FF0000"><b>Ejercicio 4.</b></span> Resolver el mismo sistema que se plantea en el ejercicio 2 mediante el método de Gauss-Seidel</b>.

In [3]:
# Ejemplo metodo de Gauss Seidel
print('Ejemplo de Gauss Seidel.')
n = 5
A = 2*eye(n) - eye(n, k=-1) - eye(n, k=1)
print('Matriz: A = ', A)
B = reshape(sum(A, axis=1), (n,1))
print('Segundo miembro: B = ', B)
X_0 = zeros((n, 1))
print('Vector inicial: X_0 = ', X_0)
exito, X = gaussseidel(A, B, X_0, 1000, 1e-5)
if exito:
    print('Solución aproximada: X_k = ', X)
    print('Comprobación: ||B-A X||_2 =',norm(B-A@X, 2))
else:
    print(X)

Ejemplo de Gauss Seidel.
Matriz: A =  [[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
Segundo miembro: B =  [[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
Vector inicial: X_0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Iteración: k =  38
Error absoluto: error =  8.277024824421275e-06
Convergencia numérica alcanzada: gausseidel.
Solución aproximada: X_k =  [[0.99998345]
 [0.99997517]
 [0.99997517]
 [0.99998138]
 [0.99999069]]
Comprobación: ||B-A X||_2 = 1.3608408022659568e-05


Finalmente la familia de <b>métodos iterativos de relajación</b> se basa en la elección $M=\frac{1}{\omega}\,D-E$ y $N=\frac{1-\omega}{\omega}\,D+F$, donde $\omega\in\mathbb{R}-\{0\}$, con lo que se tiene que $C=\left(\frac{1}{\omega}\,D-E\right)^{-1}\,\left(\frac{1-\omega}{\omega}\,D+F\right)$ (esta matriz se suele notar mediante $\mathcal{L}_\omega$) y $V=\left(\frac{1}{\omega}\,D-E\right)^{-1}\,B$. La sucesión generada $\{X_k\}_{k\in\mathbb{N}}$ viene dada por
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ X_{k+1} = \left(\frac{1}{\omega}\,D-E\right)^{-1}\,\left(\frac{1-\omega}{\omega}\,D+F\right)\,X_k + \left(\frac{1}{\omega}\,D-E\right)^{-1}\,B\,, \quad k=0,1,2,\ldots, \end{array} \right.
$$
o equivalentemente
$$
\left\{ \begin{array}{l} X_0 \in \mathbb{K}^n \quad dado\,, \\ \frac{1}{\omega}\,D\,X_{k+1} = B + E\,X_{k+1} + \frac{1-\omega}{\omega}\,D\,X_k + F\,X_k\,, \quad k=0,1,2,\ldots. \end{array} \right.
$$
<br>

Si ponemos $X_k=(x_i^{(k)})_{k=1}^n$, entonces
$$
x_i^{(k+1)} = \frac{\omega}{a_{i,i}}\,\left( b_i - \sum_{j=1}^{i-1} a_{i,j}\,x_j^{(k+1)} + \frac{1-\omega}{\omega}\,a_{i,i}\,x_i^{(k)} - \sum_{j=i+1}^n a_{i,j}\,x_j^{(k)} \right)\,.
$$

<span style="color:#FF0000"><b>Ejercicio 5.</b></span> Elaborar un programa de nombre <span style="font-family: Courier,monospace">relajacion()</span> que implemente el algoritmo del <b>método iterativo de relajación</b>.

In [4]:
# En algoritmos.py
# Ejemplo metodo de Gauss Seidel
print('Ejemplo de relajacion.')
n = 5
A = 2*eye(n) - eye(n, k=-1) - eye(n, k=1)
print('Matriz: A = ', A)
B = reshape(sum(A, axis=1), (n,1))
print('Segundo miembro: B = ', B)
X_0 = zeros((n, 1))
print('Vector inicial: X_0 = ', X_0)
exito, X = relajacion(A, B, X_0, 1.5, 1000, 1e-5)
if exito:
    print('Solución aproximada: X_k = ', X)
    print('Comprobación: ||B-A X||_2 =',norm(B-A@X, 2))
else:
    print(X)

Ejemplo de relajacion.
Matriz: A =  [[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
Segundo miembro: B =  [[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
Vector inicial: X_0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Iteración: k =  18
Error: error =  8.704980579921084e-06
Convergencia numérica alcanzada: relajación.
Solución aproximada: X_k =  [[0.99999394]
 [0.99999118]
 [0.99999404]
 [1.00000019]
 [0.99999895]]
Comprobación: ||B-A X||_2 = 1.063862396477452e-05


<span style="color:#FF0000"><b>Ejercicio 6.</b></span> Resolver el mismo sistema que se plantea en el ejercicio 2 mediante el método de relajación, tomando diferentes valores del parámetro $\omega$</b>.

In [5]:
# En algoritmos.py
# Ejemplo metodo de Gauss Seidel
print('Ejemplo de relajacion.')
n = 5
A = 2*eye(n) - eye(n, k=-1) - eye(n, k=1)
print('Matriz: A = ', A)
B = reshape(sum(A, axis=1), (n,1))
print('Segundo miembro: B = ', B)
X_0 = zeros((n, 1))
print('Vector inicial: X_0 = ', X_0)
exito, X = relajacion(A, B, X_0, 3, 1000, 1e-5)
if exito:
    print('Solución aproximada: X_k = ', X)
    print('Comprobación: ||B-A X||_2 =',norm(B-A@X, 2))
else:
    print(X)

Ejemplo de relajacion.
Matriz: A =  [[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
Segundo miembro: B =  [[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
Vector inicial: X_0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Iteración: k =  1000
Error: error =  1.07748726237158e+301
ERROR relajación: no se resuelve el sistema.


In [6]:
# En algoritmos.py
# Ejemplo metodo de Gauss Seidel
print('Ejemplo de relajacion.')
n = 5
A = 2*eye(n) - eye(n, k=-1) - eye(n, k=1)
print('Matriz: A = ', A)
B = reshape(sum(A, axis=1), (n,1))
print('Segundo miembro: B = ', B)
X_0 = zeros((n, 1))
print('Vector inicial: X_0 = ', X_0)
exito, X = relajacion(A, B, X_0, 0.5, 1000, 1e-5)
if exito:
    print('Solución aproximada: X_k = ', X)
    print('Comprobación: ||B-A X||_2 =',norm(B-A@X, 2))
else:
    print(X)

Ejemplo de relajacion.
Matriz: A =  [[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]
Segundo miembro: B =  [[1.]
 [0.]
 [0.]
 [0.]
 [1.]]
Vector inicial: X_0 =  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Iteración: k =  105
Error: error =  9.351451952466228e-06
Convergencia numérica alcanzada: relajación.
Solución aproximada: X_k =  [[0.99994565]
 [0.99991001]
 [0.99990066]
 [0.99991775]
 [0.9999546 ]]
Comprobación: ||B-A X||_2 = 4.694630387629442e-05


<span style="color:#FF0000"><b>Ejercicio 7.</b></span> Para un caso concreto de matriz tridiagonal definida positiva, calcular el parámetro óptimo de relajación, y realizar diferentes ensayos de dicho método, utilizando el valor óptimo de $\omega$, así como valores inferiores y superiores.

<span style="color:#FF0000"><b>Ejercicio 8.</b></span> En el caso de matrices tridiagonales y, en general, banda con semianchura de banda $p$, optimizar los programas anteriores de manera que se eviten las operaciones innecesarias.